In [2]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('ZHIPU_API_KEY')
base_url = "https://open.bigmodel.cn/api/paas/v4/"
chat_model = "glm-4-flash"
emb_model = "embedding-2"

# 配置对话模型
from llama_index.llms.zhipuai import ZhipuAI
llm = ZhipuAI(
    api_key = api_key,
    model = chat_model,
)

# 配置嵌入模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding
embedding = ZhipuAIEmbedding(
    api_key = api_key,
    model = emb_model,
)
emb = embedding.get_text_embedding("你好呀呀")

# 从指定文件读取，输入为List
from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['./docs/问答手册.txt']).load_data()

# 构建向量索引
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents,embed_model=embedding)
# 想要看到进度条的话，加一个参数 show_progress=True
# index = VectorStoreIndex.from_documents(documents,embed_model=embedding,show_progress=True)

我们希望做一个流式输出的后端，然后让前端去捕获这个流式输出，并且在聊天界面中流式输出。
首先构造流式输出引擎。

In [3]:
# 构造流式输出引擎
query_engine = index.as_query_engine(
    streaming=True, 
    similarity_top_k=3,
    llm=llm)

然后生成response_stream，这个response_stream里面有一个生成器，叫做response_gen。我们可以像列表一样去迭代这个生成器，然后取出生成的文本。

In [4]:
response_stream = query_engine.query("请写一篇1000字的文章论述法学专业就业前景") 
for text in response_stream.response_gen:
    print(text,end="")

随着科技的飞速发展，人工智能、大数据、云计算等新兴技术逐渐渗透到各行各业，法学专业作为一门传统学科，其就业前景也面临着前所未有的机遇与挑战。本文将从以下几个方面论述法学专业就业前景。

一、法律服务需求持续增长

随着我国法治建设的不断推进，法律服务需求持续增长。一方面，企业、个人在法律事务上的需求日益增多，对专业法律人才的需求也日益旺盛；另一方面，随着国际交流合作的加深，涉外法律服务需求也在不断上升。法学专业毕业生在法律服务领域具有明显的竞争优势。

二、新兴领域人才需求旺盛

随着新兴领域的不断涌现，如知识产权、网络安全、环境资源、金融科技等，这些领域对法学专业人才的需求也在不断增加。法学专业毕业生在这些领域具备扎实的法律功底和专业知识，能够为企业提供专业的法律服务。

三、复合型人才更受青睐

在当前就业市场中，具备跨学科知识的复合型人才更受企业青睐。法学专业毕业生可以通过学习其他领域的知识，如经济、金融、计算机等，提高自身的综合素质，成为具备跨学科能力的复合型人才。这类人才在就业市场上具有更高的竞争力。

四、公务员、事业单位就业机会丰富

法学专业毕业生在公务员、事业单位等领域具有广泛的就业机会。近年来，我国政府高度重视法治建设，加大了法治人才培养力度。法学专业毕业生在公务员、事业单位等领域具有明显的优势，就业前景广阔。

五、国际化发展前景广阔

随着我国国际地位的不断提升，涉外法律服务需求不断增加。法学专业毕业生具备较强的国际视野和跨文化沟通能力，能够在国际舞台上发挥重要作用。此外，法学专业毕业生还可以通过考取国际律师资格，从事国际法律服务。

六、法律职业资格考试通过率提高

近年来，我国法律职业资格考试通过率逐年提高，为法学专业毕业生提供了更多就业机会。通过法律职业资格考试，毕业生可以进入律师、法官、检察官等法律职业领域，实现自身价值。

七、法律教育行业蓬勃发展

随着人们对法律意识的提高，法律教育行业蓬勃发展。法学专业毕业生可以从事法律教育、培训等工作，为社会培养更多法律人才。

总之，法学专业就业前景广阔，但同时也面临着一定的挑战。法学专业毕业生应不断提升自身综合素质，关注新兴领域，拓宽就业渠道，以适应不断变化的就业市场。同时，学校、企业和社会也应共同努力，为法学专业毕业生提供更多就业机会，助力法学专业毕业生实现人生价值。

这样我们就可以在Jupyter的界面看到流式输出了。

我们也可以做一个后端，可以用FastAPI来做。这是一个新的python web框架，性能很强悍。

我们先来安装一下这个框架：
在Jupyter的格子中输入以下内容。

In [5]:
%pip install fastapi
%pip install uvicorn

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.


框架都很小，很快就安装好。

下面我们来改造一下，用fastapi做成http接口。
新建一个Jupyter notebook的格子，填入以下代码：

In [8]:
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
import threading

app = FastAPI()
app.add_middleware(CORSMiddleware,allow_origins=["*"])

# 新增服务器控制相关代码
_server_thread = None

def run_server():
    config = uvicorn.Config(app, host='0.0.0.0', port=5000)
    server = uvicorn.Server(config)
    server.run()

def start_server():
    """启动后台服务"""
    global _server_thread
    if not _server_thread or not _server_thread.is_alive():
        _server_thread = threading.Thread(target=run_server, daemon=True)
        _server_thread.start()
        print("服务已启动：http://localhost:5000/stream_chat")

@app.get('/stream_chat')
async def stream_chat(param:str = "你好"):
    async def generate():  
        response_stream = query_engine.query(param)
        for text in response_stream.response_gen:
            yield text
    return StreamingResponse(generate(), media_type='text/event-stream')

# 在Notebook中直接调用启动服务
start_server()

服务已启动：http://localhost:5000/stream_chat


INFO:     Started server process [59881]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59412 - "GET /stream_chat?param=%E6%B3%95%E5%AD%A6%E4%B8%93%E4%B8%9A%E5%B0%B1%E4%B8%9A%E5%89%8D%E6%99%AF%E5%A6%82%E4%BD%95%EF%BC%9F HTTP/1.1" 200 OK


In [9]:
# 调用服务
import requests

def test_stream_chat(question="你好"):
    url = "http://localhost:5000/stream_chat"
    params = {"param": question}
    
    with requests.get(url, params=params, stream=True) as response:
        for chunk in response.iter_content(decode_unicode=True):
            if chunk:
                print(chunk, end="", flush=True)

# 调用示例
test_stream_chat("法学专业就业前景如何？")

法学专业的就业前景广阔，尤其是在当前信息化、智能化快速发展的背景下。随着人工智能、大数据等技术的应用，法律行业对专业人才的需求日益增长。例如，在人工智能领域，法学专业毕业生可以参与到智能法律咨询、合同审查、知识产权保护等方面的工作。同时，随着法治建设的不断加强，法律服务的需求也在扩大，为法学专业毕业生提供了更多的就业机会。然而，法学专业毕业生也需要关注行业动态，不断提升自己的专业能力和综合素质，以适应不断变化的市场需求。


可以看到，这里的关键的关键在于yield语句的使用以及用一个generate函数构建Response。

然后在前端我们就可以愉快地接收了。

当然也可以新建一个python文件，填入以下代码：

```
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
app = FastAPI()
app.add_middleware(CORSMiddleware,allow_origins=["*"])
@app.get('/stream_chat')
async def stream_chat(param:str = "你好"):
    def generate():  
        # 我们假设query_engine已经构建完成
        response_stream = query_engine.query(param) 
        for text in response_stream.response_gen:
            yield text
    return StreamingResponse(generate(), media_type='text/event-stream')  
if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=5000)
```


可以看到在jupyter的格子中运行的代码与在py文件中运行的代码的区别仅仅在于`if __name__ == '__main__':`之后的代码。这是因为Jupyter是一个交互式的环境，它会在每个单元格中运行代码，而不是在一个独立的程序中运行。因此，当我们在Jupyter中运行一个程序时，它会在一个新的进程中运行，而不是在当前进程中运行。这就是为什么我们在Jupyter中运行的代码需要在`if __name__ == '__main__':`之后的代码中用uvicorn.Server启动服务器。而在py文件中运行的代码则不需要。

我们甚至可以直接在浏览器地址栏里输入：
http://127.0.0.1:5000/stream_chat?param=你是谁？
然后浏览器页面就会出现流式输出。


我们也可以把它封装到一个js函数中
```javascript
async function fetchStream(param) {  
    const url = `http://127.0.0.1:5000/stream_chat?param=${encodeURIComponent(param)}`;
    const response = await fetch(url);  
    const reader = response.body.getReader();  

    while (true) {  
        const { value, done } = await reader.read();  

        if (done) {  
            // 如果没有更多的数据可读，退出循环 
            statusvue.isTalking=false;  
            break; 
        }  

        // 处理接收到的数据  
        const text = new TextDecoder("utf-8").decode(value);  
        console.log(text)
    }  
} 
```

然后我们就可以在想要用到地地方调用这个fetchStream函数了。这个函数需要一个参数，是字符串形式的。

本教程配了一个chat.html 文件，如果后端用fastapi配置好了，浏览器http://127.0.0.1:5000/stream_chat?param=你是谁？也能有内容了，就可以在本地电脑双击打开这个chat.html 文件进行对话聊天。


好了，现在我们可以从零搭建起一个RAG应用了。未来我们可以一起搭建起一套开源的 To B的RAG系统。这个RAG系统也叫wow-rag，目前计划采用的技术栈是：
前端：TS + Vue3 + Element Plus
后端：FastAPI
大模型框架：Llama-index
大模型：自塾自封装One-API服务
数据库：Qdrant
我们计划不依赖于docker。只有一个zip包，解压后双击就部署完成。
我们计划做成五个套餐版本，分别是：
1. 如果客户只有一个破电脑，上个网都卡，我们就给他部署套餐一。
2. 如果客户有一个不错的电脑，我们就给他部署套餐二。
3. 如果客户有一个不错的服务器，我们就给他部署套餐三。
4. 如果客户有一个更好的服务器，我们就给他部署套餐四。
5. 如果客户有一个更好的服务器，我们就给他部署套餐五。
如果你有兴趣参与这项计划，请发邮件 zishuco@163.com 联系自塾，加入RAG开发员团队。